In [77]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime

In [95]:
df = pd.read_csv('Subreddits.csv')
company = pd.read_csv('Company.csv')
comp = [company.iloc[i][0].split('\t')[0] for i in range(company.shape[0])]
name = [df.iloc[i][0] for i in range(df.shape[0])]

In [96]:
def getPushshiftData(query, after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

In [97]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData
    return subStats

In [98]:
def data_frame(subStats):
    Title = []
    Publish_Date = []
    for sub in subStats:
        Title.append(subStats[sub][0][1])
        Publish_Date.append(subStats[sub][0][5])
    df_mas['Title'] = Title
    df_mas['Date'] = Publish_Date
    return df_mas

In [102]:
def scrape(after, before, query, sub):
    # We need to run this function outside the loop first to get the updated after variable
    data = getPushshiftData(query, after, before, sub)
    # Will run until all posts have been gathered i.e. When the length of data variable = 0
    # from the 'after' date up until before date
    while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
        for submission in data:
            collectSubData(submission)
            #subCount+=1
        # Calls getPushshiftData() with the created date of the last submission
        print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        #update after variable to last created date of submission
        after = data[-1]['created_utc']
        #data has changed due to the new after variable provided by above code
        data = getPushshiftData(query, after, before, sub)
        return subStats
        

In [103]:
name

['wallstreetbets',
 'stocks',
 'investing',
 'pennystocks',
 'robinhood',
 'StockMarket',
 'smallstreetbets']

In [141]:
lst = []
after = "1510185600" 
before = "1636246623" 
for i in name:
    query ='amazon' 
    sub = i
    scrape(after, before, query, sub)
    lst.append(subStats)

https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1510185600&before=1636246623&subreddit=wallstreetbets
100
2018-04-25 11:56:58
https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1524682618&before=1636246623&subreddit=wallstreetbets
https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1510185600&before=1636246623&subreddit=stocks
100
2018-09-04 17:37:39
https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1536107859&before=1636246623&subreddit=stocks
https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1510185600&before=1636246623&subreddit=investing
100
2018-05-10 04:04:22
https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1525950262&before=1636246623&subreddit=investing
https://api.pushshift.io/reddit/search/submission/?title=amazon&size=1000&after=1510185600&before=1636246623&subreddit=pennystocks
100
2020-08-26 11:1

In [142]:
len(lst)

7

In [143]:
title = []
publish = []
for subStats in lst:
    for sub in subStats:
        title.append(subStats[sub][0][1])
        publish.append(subStats[sub][0][5])

In [144]:
apple = pd.DataFrame()
apple['Title'] = title
apple['Date'] = publish

In [145]:
apple.to_csv('reddit_amazon.csv')

In [140]:
company

,Company Names\tTicker
0,Apple\tAAPL
1,Cisco\tCSCO
2,Amazon\tAMZN
3,Facebook\tFB
4,Sony\tSony
5,Google\tGOOGL


In [110]:
def data_frame(subStat):
    Title = []
    Publish_Date = []
    for subStats in subStat:
        for sub in subStats:
            Title.append(subStats[sub][0][1])
            Publish_Date.append(subStats[sub][0][5])
    df_mas['Title'] = Title
    df_mas['Date'] = Publish_Date
    return df_mas

In [109]:
res = data_frame(lst)

ValueError: Length of values (28518) does not match length of index (3517)

In [105]:
len(lst)

7

In [85]:

after = "1510185600" 
before = "1636246623" 
query = "Apple" 
sub = "wallstreetbets" 
subCount = 0
subStats = {}

In [86]:
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=Apple&size=1000&after=1510185600&before=1636246623&subreddit=wallstreetbets
100
2018-04-18 05:09:00
https://api.pushshift.io/reddit/search/submission/?title=Apple&size=1000&after=1524053340&before=1636246623&subreddit=wallstreetbets
100
2018-08-04 08:14:04
https://api.pushshift.io/reddit/search/submission/?title=Apple&size=1000&after=1533395644&before=1636246623&subreddit=wallstreetbets
99
2018-10-24 23:26:32
https://api.pushshift.io/reddit/search/submission/?title=Apple&size=1000&after=1540448792&before=1636246623&subreddit=wallstreetbets
100
2018-12-19 20:18:11
https://api.pushshift.io/reddit/search/submission/?title=Apple&size=1000&after=1545279491&before=1636246623&subreddit=wallstreetbets
100
2019-03-09 16:35:38
https://api.pushshift.io/reddit/search/submission/?title=Apple&size=1000&after=1552178138&before=1636246623&subreddit=wallstreetbets
100
2019-05-22 09:34:39
https://api.pushshift.io/reddit/search/submission/?title=Ap

In [87]:
subStats

{'7br9y7': [('7br9y7',
   'How do I inverse apple and amazon?',
   'https://www.reddit.com/r/wallstreetbets/comments/7br9y7/how_do_i_inverse_apple_and_amazon/',
   'SainteDeus',
   6,
   datetime.datetime(2017, 11, 8, 21, 15, 16),
   15,
   '/r/wallstreetbets/comments/7br9y7/how_do_i_inverse_apple_and_amazon/',
   'NaN')],
 '7fntvv': [('7fntvv',
   'investing GENIUS (autistic faggot) makes THOUSANDS on Apple, Nike, without even tucking tie into vest.',
   'https://www.reddit.com/r/wallstreetbets/comments/7fntvv/investing_genius_autistic_faggot_makes_thousands/',
   'daveydst',
   1,
   datetime.datetime(2017, 11, 26, 8, 18, 58),
   0,
   '/r/wallstreetbets/comments/7fntvv/investing_genius_autistic_faggot_makes_thousands/',
   'NaN')],
 '7gl2sf': [('7gl2sf',
   'blaming Apple for the tekk dip today cuz they messes up that root no password zero day exploit',
   'https://www.reddit.com/r/wallstreetbets/comments/7gl2sf/blaming_apple_for_the_tekk_dip_today_cuz_they/',
   '[deleted]',
   2,


In [50]:
def data_frame(subStats):
    Title = []
    Publish_Date = []
    for sub in subStats:
        Title.append(subStats[sub][0][1])
        Publish_Date.append(subStats[sub][0][5])
    df_mas['Title'] = Title
    df_mas['Date'] = Publish_Date
    return df_mas

In [47]:
Title = []
Publish_Date = []
for sub in subStats:
    Title.append(subStats[sub][0][1])
    Publish_Date.append(subStats[sub][0][5])
df_mas = pd.DataFrame()
df_mas['Title'] = Title
df_mas['Date'] = Publish_Date

In [48]:
df_mas = pd.DataFrame()
df_mas['Title'] = Title
df_mas['Date'] = Publish_Date

In [49]:
df_mas

,Title,Date
0,How do I inverse apple and amazon?,2017-11-08 21:15:16
1,investing GENIUS (autistic faggot) makes THOUS...,2017-11-26 08:18:58
2,blaming Apple for the tekk dip today cuz they ...,2017-11-30 01:04:07
3,"Petition for Apple to repatriate cash, buy Gol...",2017-12-04 08:21:49
4,What do you guys think about Tesla stock? Im t...,2017-12-26 16:40:04
...,...,...
3512,QRVO QQORVO INC Everything I read is very good...,2021-11-04 11:00:10
3513,Made my first options call today (on Apple). A...,2021-11-04 23:56:46
3514,Is it true that Apple will acquire Peloton???,2021-11-05 00:21:40
3515,Are apple and Microsoft considered “High Risk ...,2021-11-05 05:09:15


In [60]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    #filename = input() #This asks the user what to name the file
    data_frame = pd.DataFrame(columns=["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"])   
    for sub in subStats:
        data_frame.append(subStats[sub][0])
        upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

input filename of submission file, please add .csv


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [2]:
#dataframes are loaded
df = pd.read_csv('Subreddits.csv')

In [3]:
subreddits  = [df.iloc[i][0] for i in range(df.shape[0])]

In [4]:
#importing API
api = PushshiftAPI()

In [5]:
def get_pushshift_data(data_type, **kwargs):
 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = re.get(base_url, params=payload)
    return request.json()


In [6]:
5*365

1825

In [11]:
data_type="comments"     # give me comments, use "submission" to publish something
query="python"          # Add your query
after="182d"          # Select the timeframe. Epoch value or Integer + "s,m,h,d" (i.e. "second", "minute", "hour", "day")
size=1000               # maximum 1000 comments
sort_type="score"
duration='months'       # Sort by score (Accepted: "score", "num_comments", "created_utc")
sort="desc"             # sort descending
aggs="subreddit"        #"author", "link_id", "created_utc", "subreddit"


In [12]:
get_pushshift_data(data_type=data_type,     
                   q=query,                 
                   after=duration,          
                   size=size,               
                   sort_type=sort_type,
                   sort=sort)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)